## 第9章 时序数据

In [2]:
import pandas as pd
import numpy as np

### 五、练习

【练习一】 现有一份关于某超市牛奶销售额的时间序列数据，请完成下列问题：

In [7]:
df = pd.read_csv(r'joyful-pandas-master\data\time_series_one.csv', parse_dates=['日期'])
df

,日期,销售额
0,2017-02-17,2154
1,2017-02-18,2095
2,2017-02-19,3459
3,2017-02-20,2198
4,2017-02-21,2413
...,...,...
995,2019-11-09,3022
996,2019-11-10,2961
997,2019-11-11,3984
998,2019-11-12,2799


（a）销售额出现最大值的是星期几？（提示：利用dayofweek函数）

In [8]:
df['日期'].dt.dayofweek[df['销售额'].idxmax()]

6

（b）计算除去春节、国庆、五一节假日的月度销售总额

In [9]:
holiday = pd.date_range(start='20170501', end='20170503').append(
          pd.date_range(start='20171001', end='20171007')).append(
          pd.date_range(start='20180215', end='20180221')).append(
          pd.date_range(start='20180501', end='20180503')).append(
          pd.date_range(start='20181001', end='20181007')).append(
          pd.date_range(start='20190204', end='20190224')).append(
          pd.date_range(start='20190501', end='20190503')).append(
          pd.date_range(start='20191001', end='20191007'))

In [10]:
df[~df['日期'].isin(holiday)].set_index('日期').resample('MS').sum()

,销售额
日期,
2017-02-01,31740
2017-03-01,80000
2017-04-01,74734
2017-05-01,76237
2017-06-01,80750
2017-07-01,83107
2017-08-01,85715
2017-09-01,79604
2017-10-01,61197


（c）按季度计算周末（周六和周日）的销量总额

In [11]:
df[~df['日期'].dt.dayofweek.isin([0,5])].set_index('日期').sort_index(ascending=False).resample('5D').sum()

,销售额
日期,
2017-02-17,8026
2017-02-22,10917
2017-02-27,11052
2017-03-04,6945
2017-03-09,8916
...,...
2019-10-20,12817
2019-10-25,8902
2019-10-30,11955


（d）从最后一天开始算起，跳过周六和周一，以5天为一个时间单位向前计算销售总和

In [12]:
df_temp = df[~df['日期'].dt.dayofweek.isin([5,6])].set_index('日期').iloc[::-1]
L_temp,date_temp = [],[0]*df_temp.shape[0]
for i in range(df_temp.shape[0]//5):
    L_temp.extend([i]*5)
L_temp.extend([df_temp.shape[0]//5]*(df_temp.shape[0]-df_temp.shape[0]//5*5))
date_temp = pd.Series([i%5==0 for i in range(df_temp.shape[0])])
df_temp['num'] = L_temp
result = pd.DataFrame({'5天总额':df_temp.groupby('num')['销售额'].sum().values},
                       index=df_temp.reset_index()[date_temp]['日期']).iloc[::-1]
result.head()

,5天总额
日期,
2017-02-22,9855
2017-03-01,12296
2017-03-08,13323
2017-03-15,13845
2017-03-22,11356


（e）假设现在发现数据有误，所有同一周里的周一与周五的销售额记录颠倒了，请计算2018年中每月第一个周一的销售额（如果该周没有周一或周五的记录就保持不动）

In [13]:
from datetime import datetime 
df_temp = df.copy()
df_fri = df.shift(4)[df.shift(4)['日期'].dt.dayofweek==1]['销售额']
df_mon = df.shift(-4)[df.shift(-4)['日期'].dt.dayofweek==5]['销售额']
df_temp.loc[df_fri.index,['销售额']] = df_fri
df_temp.loc[df_mon.index,['销售额']] = df_mon
df_temp.loc[df_temp[df_temp['日期'].dt.year==2018]['日期'][
        df_temp[df_temp['日期'].dt.year==2018]['日期'].apply(
        lambda x:True if datetime.strptime(str(x).split()[0],'%Y-%m-%d').weekday() == 0 
        and 1 <= datetime.strptime(str(x).split()[0],'%Y-%m-%d').day <= 7 else False)].index,:]

,日期,销售额
318,2018-01-01,2863.0
353,2018-02-05,2321.0
381,2018-03-05,2705.0
409,2018-04-02,2487.0
444,2018-05-07,3204.0
472,2018-06-04,2927.0
500,2018-07-02,2574.0
535,2018-08-06,2504.0
563,2018-09-03,2483.0
591,2018-10-01,2431.0


【练习二】 继续使用上一题的数据，请完成下列问题：

（a）以50天为窗口计算滑窗均值和滑窗最大值（min_periods设为1）

In [17]:
df = pd.read_csv(r'joyful-pandas-master\data\time_series_one.csv',index_col='日期',parse_dates=['日期'])
df['销售额'].rolling(window=50,min_periods=1).max()

日期
2017-02-17    2154.0
2017-02-18    2154.0
2017-02-19    3459.0
2017-02-20    3459.0
2017-02-21    3459.0
               ...  
2019-11-09    4048.0
2019-11-10    4048.0
2019-11-11    4048.0
2019-11-12    4048.0
2019-11-13    4048.0
Name: 销售额, Length: 1000, dtype: float64

（b）现在有如下规则：若当天销售额超过向前5天的均值，则记为1，否则记为0，请给出2018年相应的计算结果

In [18]:
def f(x):
    if len(x) == 6:
        return 1 if x[-1]>np.mean(x[:-1]) else 0
    else:
        return 0
result_b = df.loc[pd.date_range(start='20171227',end='20181231'),:].rolling(
                                                    window=6,min_periods=1).agg(f)[5:].head()

（c）将(c)中的“向前5天”改为“向前非周末5天”，请再次计算结果

In [19]:
def f(x):
    if len(x) == 8:
        return 1 if x[-1]>np.mean(x[:-1][pd.Series([
            False if i in [5,6] else True for i in x[:-1].index.dayofweek],index=x[:-1].index)]) else 0
    else:
        return 0
result_c = df.loc[pd.date_range(start='20171225',end='20181231'),:].rolling(
                                    window=8,min_periods=1).agg(f)[7:].head()

    抄一次答案打卡，今天太累了，休息一晚，后面有时间再回头复习下这一章内容。